In [136]:
import sys
sys.path.append("..")

import pyodbc
from Derivados.DerivadosTipos.DerivadosSCC import *
import datetime
import pandas as pd
from Bonos.Correlaciones import ewma


In [137]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'

cnn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

In [138]:
def creacion_derivado(FechaEfectiva, FechaVenc, AjusteFeriados, NocionalActivo, MonedaActivo,\
                                                     MonedaBase, TipoTasaActivo, TipoTasaPasivo, TasaActivo,\
                                                                        TasaPasivo, FrecuenciaActivo, FrecuenciaPasivo, \
                                                                        ID, Tipo, ID_Key=None): 

    info_derivado = dict()
    info_derivado["Tipo"] = "Tipo"
    info_derivado["ID_Key"] = ""
    info_derivado["Administradora"] = "Admin"
    info_derivado["Fondo"] = "Fondo"
    info_derivado["Contraparte"] = "Contraparte"
    info_derivado["ID"] = int(ID)
    info_derivado["Nemotecnico"] = ""
    info_derivado["Mercado"] = "Local" 

    fecha = datetime.date(2017, 12, 20)
    hora = '1700'

    info_derivado["FechaEfectiva"] = str(FechaEfectiva)
    info_derivado["FechaVenc"] = str(FechaVenc)

    info_derivado["AjusteFeriados"] = str(AjusteFeriados)

    info_derivado["NocionalActivo"] = float(NocionalActivo)

    info_derivado["MonedaActivo"] = str(MonedaActivo)
    info_derivado["MonedaBase"] = str(MonedaBase)

    info_derivado["TipoTasaActivo"] = str(TipoTasaActivo)
    info_derivado["TipoTasaPasivo"] = str(TipoTasaPasivo)

    info_derivado["TasaActivo"] = float(TasaActivo)
    info_derivado["TasaPasivo"] = float(TasaPasivo)

    info_derivado["FrecuenciaActivo"] = str(FrecuenciaActivo)
    info_derivado["FrecuenciaPasivo"] = str(FrecuenciaPasivo)

    info1 = pd.DataFrame([info_derivado])
    derivado = DerivadosSCC(fecha, hora, info1, cnn)
    return derivado

derivado1 = creacion_derivado("12/11/2019", "12/11/2021", "CL", 10*(10**9), "CLP", "CLP", "Fija", "Flotante", 1.45, -1000, "Semi annual", "Semi annual", 3, "SCC")

derivado1.genera_flujos()
derivado1.valoriza_flujos()



In [139]:
def extrar_crear_derivado(ID_Key):

    derivado = ("SELECT * FROM [dbDerivados].[dbo].[TdCarteraDerivados_V2] WHERE ID_Key = '"+str(ID_Key) + "'")
    derivado = pd.read_sql(derivado, cnn)
    
    FechaEfectiva = derivado["FechaEfectiva"][0]
    FechaEfectiva = str(FechaEfectiva).split("-")
    dia = FechaEfectiva[2].split(" ")[0]
    FechaEfectiva = str(dia + "/" + str(FechaEfectiva[1]) +"/"+ str(FechaEfectiva[0]))

    FechaVenc = derivado["FechaVenc"][0]
    FechaVenc = str(FechaVenc).split("-")
    dia = FechaVenc[2].split(" ")[0]

    FechaVenc = str(dia + "/" + str(FechaVenc[1]) +"/"+ str(FechaVenc[0]))

    AjusteFeriados = derivado["AjusteFeriados"][0]
    NocionalActivo = derivado["NocionalActivo"][0]
    MonedaActivo = derivado["MonedaActivo"][0]
    MonedaBase = derivado["MonedaBase"][0]
    TipoTasaActivo = derivado["TipoTasaActivo"][0]
    TipoTasaPasivo = derivado["TipoTasaPasivo"][0]
    TasaActivo = derivado["TasaActivo"][0]
    TasaPasivo = derivado["TasaPasivo"][0]
    FrecuenciaActivo = derivado["FrecuenciaActivo"][0]
    FrecuenciaPasivo = derivado["FrecuenciaPasivo"][0]
    ID = derivado["ID"]
    Tipo = derivado["Tipo"]
    ID_Key = derivado["ID_Key"]

    dev = creacion_derivado(FechaEfectiva, FechaVenc, AjusteFeriados, NocionalActivo, MonedaActivo,\
                                                     MonedaBase, TipoTasaActivo, TipoTasaPasivo, TasaActivo,\
                                                                        TasaPasivo, FrecuenciaActivo, FrecuenciaPasivo, \
                                                                        ID, Tipo, ID_Key=ID_Key)

    return dev

C = extrar_crear_derivado("146854")
C.genera_flujos()
C.valoriza_flujos()
C.flujos_valorizados[["ID","ActivoPasivo", "Fecha", "FechaFixing", "FechaFlujo", "FechaPago", "Flujo", "ValorPresenteMonFlujo", "Moneda", "MonedaBase"]]

,ID,ActivoPasivo,Fecha,FechaFixing,FechaFlujo,FechaPago,Flujo,ValorPresenteMonFlujo,Moneda,MonedaBase
0,1310,-1,2017-12-20,2018-09-12,2018-09-12,2018-09-12,1.899260e+07,1.863861e+07,CLP,CLP
1,1310,-1,2017-12-20,2019-03-12,2019-03-12,2019-03-12,1.410621e+07,1.365073e+07,CLP,CLP
2,1310,-1,2017-12-20,2019-09-12,2019-09-12,2019-09-12,1.476790e+07,1.408308e+07,CLP,CLP
3,1310,-1,2017-12-20,2020-03-12,2020-03-12,2020-03-12,1.018056e+09,9.536276e+08,CLP,CLP
4,1310,1,2017-12-20,2018-09-12,2018-09-12,2018-09-12,1.512177e+07,1.483992e+07,CLP,CLP
5,1310,1,2017-12-20,2019-03-12,2019-03-12,2019-03-12,1.487340e+07,1.439314e+07,CLP,CLP
6,1310,1,2017-12-20,2019-09-12,2019-09-12,2019-09-12,1.512177e+07,1.442054e+07,CLP,CLP
7,1310,1,2017-12-20,2020-03-12,2020-03-12,2020-03-12,1.014956e+09,9.507236e+08,CLP,CLP


In [140]:
def seleccionar_curva_derivados(moneda):

    curva = ("SELECT TOP (1000) * FROM [dbDerivados].[dbo].[TdCurvasDerivados] WHERE Tipo = 'CurvaEfectiva_"+ moneda +"' AND Hora = '1700'")
    curva = pd.read_sql(curva, cnn)
    return curva

a = seleccionar_curva_derivados("CLP")
a

,Fecha,Hora,Tipo,Curva
0,2020-01-20,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999952044955167|7#0,9996596504..."
1,2020-01-17,1700,CurvaEfectiva_CLP,"Plazo#Valor|3#0,999873380808733|7#0,9996998671..."
2,2020-01-16,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999952145103623|7#0,9996608774..."
3,2020-01-15,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999954181295717|7#0,9996734746..."
4,2020-01-14,1700,CurvaEfectiva_CLP,"Plazo#Valor|1#0,999952232342846|7#0,9996588063..."
...,...,...,...,...
995,2016-01-20,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,9999017276407|7#0,99930821..."
996,2016-01-19,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999892448031927|7#0,999243..."
997,2016-01-18,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|1#0,999880400131654|7#0,999158..."
998,2016-01-15,1700,CurvaEfectiva_CLP,"Plazo#Valor|0#1|3#0,999814266543168|7#0,999562..."


In [141]:
vector_dias = [30, 90, 180, 360, 360*2, 360*3, 360*4, 360*5, 360*7, 360*9, 360*10, 360*15, 360*20, 360*30]


def evaluar_curva(pivotes, derivado):

    largo = len(pivotes)
    curvas = seleccionar_curva_derivados(derivado.flujos_valorizados["Moneda"][0])

    valores = []

    df = pd.DataFrame()
    df["Fechas"] = curvas["Fecha"]

    for i in range(largo):

        for j in range(len(curvas["Curva"])):

            valor_dia = pivotes[i]
            curva = curvas["Curva"][j]
            fecha_curva = curvas["Fecha"][j]
            curva_parseada = parsear_curva(curva, fecha_curva)
            valor = interpolacion_log_escalar(valor_dia, curva_parseada)
            valores.append(valor)
        
        df[str(vector_dias[i])] = valores
        valores = []

    
    return df


evaluar_curva(vector_dias, C)

,Fechas,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
0,2020-01-20,0.998486,0.995838,0.991723,0.982295,0.965200,0.943424,0.918130,0.889492,0.829008,0.763974,0.734000,0.614916,0.519875,0.374718
1,2020-01-17,0.998476,0.996076,0.992045,0.982662,0.965368,0.943788,0.918932,0.889901,0.829413,0.765411,0.735738,0.617512,0.522109,0.377115
2,2020-01-16,0.998433,0.995735,0.991830,0.981989,0.965197,0.943235,0.917672,0.887917,0.825427,0.761611,0.731738,0.612373,0.516362,0.370894
3,2020-01-15,0.998452,0.995842,0.991692,0.981892,0.964877,0.943027,0.916844,0.886522,0.823367,0.759320,0.729062,0.609334,0.509852,0.363745
4,2020-01-14,0.998425,0.995549,0.991518,0.981488,0.964466,0.941931,0.915124,0.884320,0.821113,0.757676,0.727380,0.605086,0.507603,0.361471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2016-01-20,0.996792,0.989403,0.977715,0.954987,0.925814,0.887344,0.848875,0.811048,0.735125,0.663292,0.630784,0.493794,0.378453,0.232326
996,2016-01-19,0.996538,0.989372,0.977228,0.954187,0.925453,0.886833,0.848225,0.810263,0.734056,0.661513,0.628244,0.490764,0.375367,0.229485
997,2016-01-18,0.996295,0.988852,0.977078,0.954491,0.926010,0.887550,0.849088,0.810845,0.735235,0.662876,0.630318,0.493194,0.377806,0.231723
998,2016-01-15,0.996371,0.988664,0.977145,0.954134,0.926026,0.887235,0.848947,0.810686,0.734957,0.662527,0.629871,0.492598,0.377161,0.231139


In [142]:


def calcular_retornos(dfHistorico):

    numero_filas = len(vector_dias)
    numero_columnas = len(dfHistorico["30"])
    valores = []
    df = pd.DataFrame()


    for i in range(numero_filas):

        columna = dfHistorico[str(vector_dias[i])]

        for j in range(numero_columnas):

            if j == 0:

                valores.append(0)

            else:

                valor = np.log(columna[j] / columna[j-1])
                valores.append(valor)

        df[str(vector_dias[i])] = valores
        valores = []

    return df
        
a = evaluar_curva(vector_dias, C)
b = calcular_retornos(a)
b

,30,90,180,360,720,1080,1440,1800,2520,3240,3600,5400,7200,10800
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000010,0.000240,0.000325,0.000373,0.000175,0.000386,0.000873,0.000460,0.000488,0.001879,0.002364,0.004212,0.004288,0.006377
2,-0.000043,-0.000343,-0.000217,-0.000684,-0.000177,-0.000586,-0.001373,-0.002232,-0.004818,-0.004977,-0.005451,-0.008356,-0.011068,-0.016635
3,0.000019,0.000107,-0.000139,-0.000099,-0.000332,-0.000221,-0.000902,-0.001572,-0.002498,-0.003012,-0.003664,-0.004975,-0.012686,-0.019463
4,-0.000027,-0.000294,-0.000176,-0.000411,-0.000425,-0.001162,-0.001877,-0.002487,-0.002742,-0.002168,-0.002310,-0.006996,-0.004421,-0.006272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000320,-0.000014,0.000321,0.000984,-0.000401,-0.000518,0.000150,-0.000227,-0.000163,-0.001254,0.001625,0.003112,0.004146,0.006199
996,-0.000255,-0.000032,-0.000498,-0.000838,-0.000390,-0.000576,-0.000766,-0.000968,-0.001455,-0.002685,-0.004035,-0.006156,-0.008188,-0.012301
997,-0.000244,-0.000526,-0.000154,0.000319,0.000602,0.000809,0.001017,0.000718,0.001605,0.002058,0.003296,0.004939,0.006475,0.009704
998,0.000077,-0.000190,0.000069,-0.000374,0.000018,-0.000356,-0.000166,-0.000196,-0.000378,-0.000527,-0.000708,-0.001209,-0.001707,-0.002526


In [143]:


def volatilidades_derivados(dfRetornos):

    numero_filas = len(vector_dias)
    numero_columnas = len(dfRetornos[str(vector_dias[0])])
    df = pd.DataFrame()
    df["Pivotes"] = vector_dias
    valores = []

    for i in range(numero_filas):

        retornos = dfRetornos[str(vector_dias[i])]
        valor = ewma(retornos, 0.94)
        valores.append(valor["Vol c/ajuste"].values[0])
    
    df["Volatilidades"] = valores
    return df


volatilidades_derivados(b)



,Pivotes,Volatilidades
0,30,0.000176
1,90,0.000281
2,180,0.000328
3,360,0.000777
4,720,0.000473
5,1080,0.000758
6,1440,0.000869
7,1800,0.001186
8,2520,0.001783
9,3240,0.002804
